In [9]:
from bs4 import BeautifulSoup
import os
import re

folder = "D:\\RAM\\AI\\NLP\\REP_gen\\pro_1\\pdf_analysis\\samples_pdf\\"

def extract_sections_from_html(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    
    text_blocks = soup.find_all(["h1", "h2", "h3", "p", "div"])
    sections = {}
    current_section = "Unknown"

    for tag in text_blocks:
        text = tag.get_text(strip=True)
        if not text:
            continue

        # Match section headers
        match = re.match(r"(abstract|introduction|methods?|results?|discussion|conclusion|references?)", text.lower())
        if match:
            current_section = match.group(1).capitalize()
            sections[current_section] = []
        else:
            sections.setdefault(current_section, []).append(text)

    return sections

# Loop through all HTML files and extract
for filename in os.listdir(folder):
    if filename.endswith(".html"):
        path = os.path.join(folder, filename)
        print(f"\n📄 {filename}")
        sections = extract_sections_from_html(path)

        for section, content in sections.items():
            print(f"\n🔹 {section} ({len(content)} blocks):")
            print("   " + content[0][:150] + "...")  # Preview first 150 chars of the section



📄 article_1.html

🔹 Unknown (35 blocks):
   Your privacy, your choiceWe use essential cookies to make sure the site can function. We also use optional cookies for advertising, personalisation of...

🔹 Abstract (121 blocks):
   Advertisement...

🔹 References (184 blocks):
   Ding Y, John NW, Smith L, Sun JA, Smith M (2015) Combination of 3D skin surface texture features and 2D ABCD features for improved melanoma diagnosis....

📄 article_2.html

🔹 Unknown (35 blocks):
   Your privacy, your choiceWe use essential cookies to make sure the site can function. We also use optional cookies for advertising, personalisation of...

🔹 Abstract (82 blocks):
   Advertisement...

🔹 References (139 blocks):
   Alam MM, Islam MT (2019) Machine learning approach of automatic identification and counting of blood cells. Healthc Technol Lett 6(4):103–108ArticleGo...

📄 article_3.html

🔹 Unknown (44 blocks):
   Your privacy, your choiceWe use essential cookies to make sure the site can function. We also us

In [ ]:
import ollama

# Load the .html file directly
file = "D:\\RAM\AI\\NLP\\REP_gen\\pro_1\\pdf_analysis\\samples_pdf\\real_page.html"
with open(file, "r", encoding="utf-8") as f:
    html_content = f.read()

# Initialize a conversation
response = ollama.chat(
    model="llama3.2",  # Or "mistral", "phi", etc.
    messages=[
        {"role": "system", "content": "You are an assistant who reads and explains HTML pages."},
        {"role": "user", "content": f"Here is the content of a website:\n\n{html_content}\n\nCan you summarize what this page is about?"}
    ]
)

print("🤖", response["message"]["content"])

<>:4: SyntaxWarning: invalid escape sequence '\A'
<>:4: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Admin\AppData\Local\Temp\ipykernel_11680\1582623928.py:4: SyntaxWarning: invalid escape sequence '\A'
  file = "D:\\RAM\AI\\NLP\\REP_gen\\pro_1\\pdf_analysis\\samples_pdf\\real_page.html"


🤖 This appears to be a Google Scholar search results page, but the content of the page is not visible due to its lengthy and complex HTML structure.

However, based on the structure and layout of the page, it appears that this is a search results page for academic papers related to "convolutional neural networks" (CNNs). The page displays a list of 10 research articles or papers that match the search query, along with relevant metadata such as author names, publication dates, and titles.

If you'd like to see the actual content of the pages listed on this page, you would need to click on the individual article titles to view their full texts.


In [41]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from dotenv import load_dotenv
import os
load_dotenv()

coder = AssistantAgent(
    "code_writer",
    llm_config={"config_list":[{"model": "gpt-4o","api_key": os.getenv("openai_key_2")}]},
    #llm_config={"config_list":[{"model":"codegemma","api_key":"ollama", "base_url": "http://localhost:11434"}]},
    human_input_mode="NEVER",
    system_message="""You are a helpful assistant that can write code in Python
    if the user ask some general question ohter than coding, just answer that without use executor in your general knowledge
    once done with erply send `Done` also with respon to terminate the chat"""
)

executor = UserProxyAgent(
    "user_proxy",
    human_input_mode="TERMINATE",
    is_termination_msg=lambda msg : "Done" in msg['content'],
    default_auto_reply="here is the result!!!",
    max_consecutive_auto_reply=2,
    #code_execution_config={"work_dir": "code","use_docker": False},
)

executor.initiate_chat(coder,message="Write a Python function that prints 'Hello, World!'")
#executor.initiate_chat(coder,message="what is quantum computing say that in 30 words")


user_proxy (to code_writer):

Write a Python function that prints 'Hello, World!'

--------------------------------------------------------------------------------
code_writer (to user_proxy):

Certainly! Here's a simple Python function that prints 'Hello, World!':

```python
def print_hello_world():
    print('Hello, World!')

# Call the function
print_hello_world()
```

This function, when called, will output `Hello, World!` to the console. 

Done

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (81b2f08d-36a2-4779-9f87-190a2ad6a464): Termination message condition on agent 'user_proxy' met and no human input provided


ChatResult(chat_id=None, chat_history=[{'content': "Write a Python function that prints 'Hello, World!'", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Certainly! Here's a simple Python function that prints 'Hello, World!':\n\n```python\ndef print_hello_world():\n    print('Hello, World!')\n\n# Call the function\nprint_hello_world()\n```\n\nThis function, when called, will output `Hello, World!` to the console. \n\nDone", 'role': 'user', 'name': 'code_writer'}], summary="Certainly! Here's a simple Python function that prints 'Hello, World!':\n\n```python\ndef print_hello_world():\n    print('Hello, World!')\n\n# Call the function\nprint_hello_world()\n```\n\nThis function, when called, will output `Hello, World!` to the console. \n\nDone", cost={'usage_including_cached_inference': {'total_cost': 0.0008425, 'gpt-4o-2024-08-06': {'cost': 0.0008425, 'prompt_tokens': 77, 'completion_tokens': 65, 'total_tokens': 142}}, 'usage_excluding_cached_inference': {'total_cost': 0.0008425,